In [1]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [2]:
import importlib
importlib.reload(extract)

<module 'extract' from '/Users/kguan17/Documents/GitHub/RA-Work/acs/extract.py'>

In [3]:
# open state_fips
state_fips = pd.read_csv('state_fips.csv')

In [4]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [36]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data/sf supervisor districts 2012/geo_export_402c9c93-9abd-49fc-a94c-2361f05d8cc0.shp')
districts

,supname,supdist,supdistpad,supervisor,numbertext,geometry
0,Fewer,SUPERVISORIAL DISTRICT 1,01,1.0,ONE,"POLYGON ((-122.49374 37.78761, -122.49367 37.7..."
1,Stefani,SUPERVISORIAL DISTRICT 2,02,2.0,TWO,"POLYGON ((-122.41922 37.80845, -122.41921 37.8..."
2,Peskin,SUPERVISORIAL DISTRICT 3,03,3.0,THREE,"POLYGON ((-122.39198 37.79387, -122.39218 37.7..."
3,Mar,SUPERVISORIAL DISTRICT 4,04,4.0,FOUR,"POLYGON ((-122.47485 37.76179, -122.47496 37.7..."
4,Preston,SUPERVISORIAL DISTRICT 5,05,5.0,FIVE,"POLYGON ((-122.42157 37.78662, -122.42145 37.7..."
5,Haney,SUPERVISORIAL DISTRICT 6,06,6.0,SIX,"MULTIPOLYGON (((-122.39198 37.79387, -122.3915..."
6,Yee,SUPERVISORIAL DISTRICT 7,07,7.0,SEVEN,"POLYGON ((-122.44854 37.75904, -122.44847 37.7..."
7,Mandelman,SUPERVISORIAL DISTRICT 8,08,8.0,EIGHT,"POLYGON ((-122.42327 37.77206, -122.42325 37.7..."
8,Ronen,SUPERVISORIAL DISTRICT 9,09,9.0,NINE,"POLYGON ((-122.41093 37.76941, -122.41088 37.7..."
9,Walton,SUPERVISORIAL DISTRICT 10,10,10.0,TEN,"MULTIPOLYGON (((-122.38487 37.76724, -122.3848..."


In [37]:
# only keep district and geometry
district_name = 'supdistpad'
year=2012
desc='san_francisco_city_council'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
districts['district'] = districts['district'].astype(int)

In [38]:
districts

,district,geometry
0,1,"POLYGON ((-122.49374 37.78761, -122.49367 37.7..."
1,2,"POLYGON ((-122.41922 37.80845, -122.41921 37.8..."
2,3,"POLYGON ((-122.39198 37.79387, -122.39218 37.7..."
3,4,"POLYGON ((-122.47485 37.76179, -122.47496 37.7..."
4,5,"POLYGON ((-122.42157 37.78662, -122.42145 37.7..."
5,6,"MULTIPOLYGON (((-122.39198 37.79387, -122.3915..."
6,7,"POLYGON ((-122.44854 37.75904, -122.44847 37.7..."
7,8,"POLYGON ((-122.42327 37.77206, -122.42325 37.7..."
8,9,"POLYGON ((-122.41093 37.76941, -122.41088 37.7..."
9,10,"MULTIPOLYGON (((-122.38487 37.76724, -122.3848..."


In [39]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)